### 기상청 날씨 데이터 파싱
* BeautifulSoup의 find(), find_all() 함수 사용
* 파싱한 데이터를 List, Dict 에 저장
* Json file로 저장

In [11]:
import requests
from bs4 import BeautifulSoup

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp"
res = requests.get(url)
print(res.status_code)
soup = BeautifulSoup(res.text, 'html.parser')

print('find :', type(soup.find('title')))
print(soup.find('title'))
print(soup.find('title').text)
print('------------')
print('find_all :', type(soup.find_all('title')))
print(soup.find_all('title'))
for title in soup.find_all('title'):
    print(title.text)

200
find : <class 'bs4.element.Tag'>
<title>기상청 육상 중기예보</title>
기상청 육상 중기예보
------------
find_all : <class 'bs4.element.ResultSet'>
[<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2021년 10월 07일 (목)요일 06:00 발표</title>, <title>전국 육상중기예보</title>]
기상청 육상 중기예보
전국 육상 중기예보 - 2021년 10월 07일 (목)요일 06:00 발표
전국 육상중기예보


In [14]:
print('select_one ', type(soup.select_one('title')))
print(soup.select_one('title'))
print(soup.select_one('title').text)

select_one  <class 'bs4.element.Tag'>
<title>기상청 육상 중기예보</title>
기상청 육상 중기예보


In [15]:
print('select :', type(soup.select('title')))
print(soup.select('title'))
for title in soup.select('title'):
    print(title.text)

select : <class 'bs4.element.ResultSet'>
[<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2021년 10월 07일 (목)요일 06:00 발표</title>, <title>전국 육상중기예보</title>]
기상청 육상 중기예보
전국 육상 중기예보 - 2021년 10월 07일 (목)요일 06:00 발표
전국 육상중기예보


In [29]:
location_one = soup.find('location',attrs={"wl_ver":"3"})
print(type(location_one))
province = location_one.find('province').text
print(province)
city = location_one.find('city').text
print(city)

data_all = location_one.find_all('data')
print(len(data_all))
for data in data_all:
    #print(data)
    mode = data.find('mode').text
    tmef = data.find('tmef').text
    print(mode, tmef)

<class 'bs4.element.Tag'>
서울ㆍ인천ㆍ경기도
서울
13
A02 2021-10-10 00:00
A02 2021-10-10 12:00
A02 2021-10-11 00:00
A02 2021-10-11 12:00
A02 2021-10-12 00:00
A02 2021-10-12 12:00
A02 2021-10-13 00:00
A02 2021-10-13 12:00
A02 2021-10-14 00:00
A02 2021-10-14 12:00
A01 2021-10-15 00:00
A01 2021-10-16 00:00
A01 2021-10-17 00:00


In [30]:
location_one2 = soup.select_one("location[wl_ver='3']")
#location_one2

### 전국의 날씨 데이터를 list에 dict에 저장하기

In [44]:
import requests
from bs4 import BeautifulSoup

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp"
res = requests.get(url)
if res.ok:
    print(res.status_code)
    soup = BeautifulSoup(res.text, 'html.parser')

    print(len(soup.find_all('location',attrs={"wl_ver":"3"})))
    #41개의 location 정보를 저장할 List 선언
    location_list = list()
    for location in soup.find_all('location',attrs={"wl_ver":"3"}):
        #1개의 location 정보를 저장할 Dict 선언
        location_dict = dict()
        
        location_dict['province'] = location.find('province').text
        location_dict['city'] = location.find('city').text
        
        # location 하위에 있는 13개의 data 정보를 저장할 List 선언
        data_list = []
        for data in location.find_all('data'):
            """
            <data>
                <mode>A02</mode>
                <tmef>2021-10-10 00:00</tmef>
                <wf>구름많음</wf>
                <tmn>20</tmn>
                <tmx>25</tmx>
                <reliability></reliability>
                <rnst>30</rnst>
            </data>
            """
            #1개의 data 정보를 저장할 Dict 선언
            data_dict = {}
            data_dict['mode'] = data.find('mode').text
            data_dict['tmef'] = data.find('tmef').text
            data_dict['wf'] = data.find('wf').text
            data_dict['tmn'] = data.find('tmn').text
            data_dict['tmx'] = data.find('tmx').text
            #data_dict를 data_list에 append
            data_list.append(data_dict)
        
        #data_list를 location_dict에 저장하기
        location_dict['datas'] = data_list
        #print(location_dict)
        
        #location_dict를 location_list에 append
        location_list.append(location_dict)

print(len(location_list))        
print(location_list[0:2])

200
41
41
[{'province': '서울ㆍ인천ㆍ경기도', 'city': '서울', 'datas': [{'mode': 'A02', 'tmef': '2021-10-10 00:00', 'wf': '구름많음', 'tmn': '20', 'tmx': '25'}, {'mode': 'A02', 'tmef': '2021-10-10 12:00', 'wf': '구름많고 비', 'tmn': '20', 'tmx': '25'}, {'mode': 'A02', 'tmef': '2021-10-11 00:00', 'wf': '구름많음', 'tmn': '14', 'tmx': '22'}, {'mode': 'A02', 'tmef': '2021-10-11 12:00', 'wf': '맑음', 'tmn': '14', 'tmx': '22'}, {'mode': 'A02', 'tmef': '2021-10-12 00:00', 'wf': '맑음', 'tmn': '14', 'tmx': '24'}, {'mode': 'A02', 'tmef': '2021-10-12 12:00', 'wf': '맑음', 'tmn': '14', 'tmx': '24'}, {'mode': 'A02', 'tmef': '2021-10-13 00:00', 'wf': '맑음', 'tmn': '15', 'tmx': '25'}, {'mode': 'A02', 'tmef': '2021-10-13 12:00', 'wf': '맑음', 'tmn': '15', 'tmx': '25'}, {'mode': 'A02', 'tmef': '2021-10-14 00:00', 'wf': '맑음', 'tmn': '15', 'tmx': '24'}, {'mode': 'A02', 'tmef': '2021-10-14 12:00', 'wf': '맑음', 'tmn': '15', 'tmx': '24'}, {'mode': 'A01', 'tmef': '2021-10-15 00:00', 'wf': '구름많음', 'tmn': '15', 'tmx': '20'}, {'mode': 'A01', 

### Json 파일로 저장하고 읽어오기

In [46]:
import json

with open('data/weather.json','w', encoding='utf8') as file:
    json.dump(location_list, file)

In [49]:
with open('data/weather.json','r',encoding='utf8') as file:
    content = file.read()
    json_content = json.loads(content)

json_content

[{'province': '서울ㆍ인천ㆍ경기도',
  'city': '서울',
  'datas': [{'mode': 'A02',
    'tmef': '2021-10-10 00:00',
    'wf': '구름많음',
    'tmn': '20',
    'tmx': '25'},
   {'mode': 'A02',
    'tmef': '2021-10-10 12:00',
    'wf': '구름많고 비',
    'tmn': '20',
    'tmx': '25'},
   {'mode': 'A02',
    'tmef': '2021-10-11 00:00',
    'wf': '구름많음',
    'tmn': '14',
    'tmx': '22'},
   {'mode': 'A02',
    'tmef': '2021-10-11 12:00',
    'wf': '맑음',
    'tmn': '14',
    'tmx': '22'},
   {'mode': 'A02',
    'tmef': '2021-10-12 00:00',
    'wf': '맑음',
    'tmn': '14',
    'tmx': '24'},
   {'mode': 'A02',
    'tmef': '2021-10-12 12:00',
    'wf': '맑음',
    'tmn': '14',
    'tmx': '24'},
   {'mode': 'A02',
    'tmef': '2021-10-13 00:00',
    'wf': '맑음',
    'tmn': '15',
    'tmx': '25'},
   {'mode': 'A02',
    'tmef': '2021-10-13 12:00',
    'wf': '맑음',
    'tmn': '15',
    'tmx': '25'},
   {'mode': 'A02',
    'tmef': '2021-10-14 00:00',
    'wf': '맑음',
    'tmn': '15',
    'tmx': '24'},
   {'mode': 'A02',
   